In [1]:
from cnns_classes import t1_quantizedCNN, t2_quantizedCNN
from utils import utils

In [2]:
#https://www.kaggle.com/code/vmarkin/advatt
# carregar as bibliotecas básicas necessárias
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms as T
from utils import utils
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.metrics import accuracy_score

# gerar os gráficos logo abaixo dos comandos de plot
%matplotlib inline
torch.manual_seed(4)

In [3]:
batch_size = 512

topologies = [
    {
        'id':1, 
        'tp_class':t1_quantizedCNN,
        'quant': [2,4,8]
    },
    {
        'id':2, 
        'tp_class':t2_quantizedCNN,
        'quant': [2,4,8]
    }
]

target_fps_list = [500,5000,50000]

device = torch.device('cpu')

In [4]:
class SatImgDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
        self.transform = T.ToTensor()
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, index):
        x = self.transform(self.X[index])
        y = torch.FloatTensor(self.y[index])
        return {'x':x, 'y':y}

In [6]:
def arraysCorrectness(a,b):
    if(len(a) == len(b)):
        sucess = 0
        for i in range(len(a)):
            if (int(a[i]) == b[i]):
                sucess = sucess + 1            
    
        correctness = sucess/len(a)
        return correctness

In [7]:
#checking if the outputs are the same...they are!!!
for tp in topologies:
    for quant in tp['quant']:
        hardware_config_name = f"t{tp['id']}w{quant}"
        ! diff -s ./pynq_results/runs/{hardware_config_name}/hardware_output_{hardware_config_name}_500fps.npy ./pynq_results/runs/{hardware_config_name}/hardware_output_{hardware_config_name}_5000fps.npy
        ! diff -s ./pynq_results/runs/{hardware_config_name}/hardware_output_{hardware_config_name}_500fps.npy ./pynq_results/runs/{hardware_config_name}/hardware_output_{hardware_config_name}_50000fps.npy

Files ./pynq_results/runs/t1w2/hardware_output_t1w2_500fps.npy and ./pynq_results/runs/t1w2/hardware_output_t1w2_5000fps.npy are identical
Files ./pynq_results/runs/t1w2/hardware_output_t1w2_500fps.npy and ./pynq_results/runs/t1w2/hardware_output_t1w2_50000fps.npy are identical
Files ./pynq_results/runs/t1w4/hardware_output_t1w4_500fps.npy and ./pynq_results/runs/t1w4/hardware_output_t1w4_5000fps.npy are identical
Files ./pynq_results/runs/t1w4/hardware_output_t1w4_500fps.npy and ./pynq_results/runs/t1w4/hardware_output_t1w4_50000fps.npy are identical
Files ./pynq_results/runs/t1w8/hardware_output_t1w8_500fps.npy and ./pynq_results/runs/t1w8/hardware_output_t1w8_5000fps.npy are identical
Files ./pynq_results/runs/t1w8/hardware_output_t1w8_500fps.npy and ./pynq_results/runs/t1w8/hardware_output_t1w8_50000fps.npy are identical
Files ./pynq_results/runs/t2w2/hardware_output_t2w2_500fps.npy and ./pynq_results/runs/t2w2/hardware_output_t2w2_5000fps.npy are identical
Files ./pynq_results/run

In [30]:
outputs = []

for tp in topologies:
    for quant in tp['quant']:
            if quant == 2:
                continue
            hardware_config_name = f"t{tp['id']}w{quant}"
            hardware_outputs = np.load(f"./pynq_results/runs/{hardware_config_name}/hardware_output_{hardware_config_name}_500fps.npy")
            outputs.append(hardware_outputs)

imgs_id = []
imgs_hash = [[],[],[],[],[],[]]

for i in range(len(outputs[0])):
    matches = 0
    for j in range(len(outputs)):
        if outputs[0][i] == outputs[j][i]:
            matches += 1
        
    if matches != len(outputs):
        continue
    else:
        imgs_id.append(i)
        imgs_hash[int(outputs[0][i])].append(i)
        

#testing
i = 0;
for id_list in imgs_hash:
    print(f"priting for class {i}")
    print(f"{id_list}");
    print(f"")    
    i+=1

priting for class 0
[18, 19, 106, 241, 297, 365, 400, 429, 455, 600, 673, 676, 685, 764, 766, 876, 891, 927, 939, 965, 995]

priting for class 1
[3, 7, 22, 24, 27, 29, 35, 49, 51, 60, 66, 67, 74, 75, 83, 91, 96, 97, 104, 107, 111, 112, 115, 117, 118, 123, 127, 141, 142, 150, 153, 154, 155, 157, 161, 166, 174, 177, 181, 182, 185, 197, 198, 202, 204, 205, 208, 211, 212, 213, 214, 219, 223, 226, 227, 232, 236, 243, 245, 248, 249, 255, 274, 278, 279, 292, 296, 301, 302, 307, 312, 317, 318, 323, 336, 340, 341, 343, 361, 362, 369, 377, 381, 386, 388, 397, 401, 408, 415, 417, 424, 425, 442, 456, 459, 466, 469, 479, 482, 485, 488, 490, 495, 497, 500, 502, 503, 514, 519, 520, 523, 531, 534, 545, 547, 549, 554, 560, 562, 565, 566, 568, 581, 587, 594, 617, 621, 622, 630, 639, 646, 648, 650, 652, 661, 666, 670, 672, 679, 690, 694, 695, 700, 701, 707, 709, 711, 713, 715, 717, 728, 732, 733, 735, 742, 743, 750, 753, 754, 757, 765, 771, 773, 774, 776, 778, 789, 794, 798, 806, 810, 816, 830, 836, 837,

/tmp/ipykernel_1588/300459336.py:24: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  imgs_hash[int(outputs[0][i])].append(i)


In [13]:
#TODO: PYNQ outputs